In [1]:
import osmnx as ox
import pandas as pd
import numpy as np
from Data import Data 
import networkx as nx
from networkx import algorithms 
import math

In [2]:
graph = ox.graph_from_address('Athens, Municipality of Athens, Regional Unit of Central Athens, Attica, 10667, Greece', network_type='drive') 

In [5]:
df = pd.read_pickle("sample.pkl")
X = df.copy()

In [35]:
EARTH_RADIUS = 6373

def euc_dist(coord0, lon1, lat1):
    lat0, lon0 = coord0
    lat1, lon1 = coord1

    lat0, lon0 = math.radians(lat0), math.radians(lon0)
    lat1, lon1 = math.radians(lat1), math.radians(lon1)

    dlat = lat1 - lat0
    dlon = lon1 - lon0

    a = math.sin(dlat/2)**2 + math.cos(lat0) * math.cos(lat1) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return EARTH_RADIUS*c

In [36]:
def edge_progress(edge_start_node, edge_end_node, v_coord):
    start_coord = graph.nodes[edge_start_node]['y'], graph.nodes[edge_start_node]['x']
    end_coord = graph.nodes[edge_end_node]['y'], graph.nodes[edge_end_node]['x']

    a = Distance_Euc(start_coord, end_coord)
    b = Distance_Euc(start_coord, v_coord)
    return b/a

In [50]:
def construct_graph_data_cols(row):
    coord = (row['lat'],row['lon'])
    nn = ox.get_nearest_node(graph, coord, method='euclidean')
    start, end, _ = ox.get_nearest_edge(graph, coord)
    edge_prog = edge_progress(start, end, coord)
    return nn, start, end, edge_prog

df_test = df[:10]
zipped_cols = zip(*df_test.apply(construct_graph_data_cols,axis=1))
df_test['nearest_node'], \
df_test['nearest_edge_start_node'], \
df_test['nearest_edge_end_node'], \
df_test['edge_progress'] \
    = zipped_cols

df_test

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                      
1  0.00  37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04  37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08  37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12  37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16  37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   
   0.20  37.977496  23.735393  34.0409   0.0906   0.0131  Taxi      320.26   
   0.24  37.977499  23.735390  34.0574   0.1390   0.0142  Taxi      320.26   
   0.28  37.977502  23.735388  34.0805   0.1816   0.0153  Taxi      320.26   
   0.32  37.977505  23.735386  34.1089   0.2123   0.0164  Taxi      320.26   
   0.36  37.977508  23.735383  34.1414   0.2398   0.0175  Taxi      320.26   

         avg_speed  nearest_node  nearest_edge_start_node  \
id time                                                     
1  0.00  35.366264    3339821648               3339821648   
   0.04  35.366264    3339821648               3339821648   
   0.08  35.366264    3339821648               3339821648   
   0.12  35.366264    3339821648               3339821648   
   0.16  35.366264    3339821648               3339821648   
   0.20  35.366264    3339821648               3339821648   
   0.24  35.366264    3339821648               3339821648   
   0.28  35.366264     250691723               3339821648   
   0.32  35.366264     250691723               3339821648   
   0.36  35.366264     250691723               3339821648   

         nearest_edge_end_node  edge_progress  
id time                                        
1  0.00              250691723       0.488886  
   0.04              250691723       0.491508  
   0.08              250691723       0.493838  
   0.12              250691723       0.496168  
   0.16              250691723       0.498790  
   0.20              250691723       0.500538  
   0.24              250691723       0.503160  
   0.28              250691723       0.505490  
   0.32              250691723       0.507820  
   0.36              250691723       0.510442